In [ ]:
#If you want to load your dataset from google drive, uncomment the following codes

from google.colab import files
from google.colab import drive
#Mounting Google Drive
drive.mount('/content/drive')
#%unload_ext google.colab.data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew
%matplotlib inline
sns.set()
import matplotlib as mpl
mpl.style.use('classic')
params = {'mathtext.default': 'regular' }
plt.rcParams.update(params)
mpl.rcParams['font.size'] = 8

#Loading OP DataSet from Google Drive
df = pd.read_excel('/content/drive/MyDrive/Model Data.xlsx') #Loaded from my own google drive
df.head()

,Date,7-day Ozone Concentration,7-day PM2.5 Concentration,7-day PM10 Concentration,Daily Average Dry Bulb Temperature,Daily Average Relative Humidity,Daily Average Wind Speed,Daily Peak Wind Direction,Daily Precipitation,Daily Max 8-hour Ozone Concentration,Daily Average PM2.5 Concentration,Daily Average PM10 Concentration
0,2015-01-08,0.022429,8.500787,19.839286,5,74,17.6,300,0.06,0.026400,6.260000,34.000000
1,2015-01-09,0.022600,8.580787,21.696429,5,74,17.6,300,0.06,0.030778,6.292000,21.500000
2,2015-01-10,0.024497,8.033215,21.767857,11,67,11.7,190,0.00,0.027200,7.360000,29.000000
3,2015-01-11,0.027954,7.520048,23.053571,27,71,8.3,180,0.19,0.021444,8.110000,24.000000
4,2015-01-12,0.026917,7.410762,24.125000,27,71,8.3,180,0.19,0.032000,9.866667,20.857143


In [ ]:
#Separating the input and output variables
X = df.drop(columns = [ "Daily Max 8-hour Ozone Concentration", "Daily Average PM2.5 Concentration", "Daily Average PM10 Concentration"], axis = 1)
O3 = df[ "Daily Max 8-hour Ozone Concentration"]
PM25 = df[ "Daily Average PM2.5 Concentration"]
PM10 = df["Daily Average PM10 Concentration"]

from sklearn.model_selection import train_test_split
seed = 42 #Seed value for reproducibility

#Train-test split [K-fold]
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,random_state=seed,shuffle=True)

# for train_index , test_index in kf.split(X):
#     X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
#     O3_train , O3_test = O3[train_index] , O3[test_index]
#     PM25_train , PM25_test = PM25[train_index] , PM25[test_index]
#     PM10_train , PM10_test = PM10[train_index] , PM10[test_index]

#Train-test split [70:30]
X_train, X_test, O3_train, O3_test = train_test_split(X, O3, test_size = 0.3, random_state = seed)
X_train1, X_test1, PM25_train, PM25_test = train_test_split(X, PM25, test_size = 0.3, random_state = seed)
X_train2, X_test2, PM10_train, PM10_test = train_test_split(X, PM10, test_size = 0.3, random_state = seed)

In [ ]:
from numpy.core.multiarray import concatenate
TrainDate = X_train['Date']
TestDate = X_test['Date']
TestDate

2008   2020-08-27
1041   2017-12-01
2358   2021-08-25
2164   2021-02-12
1641   2019-08-14
          ...    
2372   2021-09-08
911    2017-07-24
449    2016-04-09
2035   2020-09-23
1362   2018-10-28
Name: Date, Length: 876, dtype: datetime64[ns]

In [ ]:

TargetO3 = pd.DataFrame(columns = ['Date', 'Actual'])
TargetO3['Date'] = concatenate((TrainDate,TestDate), axis=0)
TargetO3['Actual'] = concatenate((O3_train,O3_test), axis=0)
TargetPM25 = pd.DataFrame(columns = ['Date', 'Actual'])
TargetPM25['Date'] = concatenate((TrainDate,TestDate), axis=0)
TargetPM25['Actual'] = concatenate((PM25_train,PM25_test), axis=0)
TargetPM10 = pd.DataFrame(columns = ['Date', 'Actual'])
TargetPM10['Date'] = concatenate((TrainDate,TestDate), axis=0)
TargetPM10['Actual'] = concatenate((PM10_train,PM10_test), axis=0)
X_train_1 = X_train.drop('Date',axis=1)
X_train_1
X_test_1 = X_test.drop('Date',axis=1)
TargetO3

,Date,Actual
0,2018-01-10,0.030000
1,2015-04-02,0.032227
2,2021-10-05,0.036429
3,2017-03-11,0.038611
4,2017-05-15,0.056053
...,...,...
2913,2021-09-08,0.044143
2914,2017-07-24,0.056286
2915,2016-04-09,0.030524
2916,2020-09-23,0.018050


In [ ]:
#Importing packages

from sklearn.preprocessing import MinMaxScaler

#Scaling
X_train_scaled = MinMaxScaler().fit_transform(X_train_1) #x_norm = (x-xmin)/(xmax-xmin) #x_norm= (x-xmean)/ x_std
X_test_scaled = MinMaxScaler().fit_transform(X_test_1)

In [ ]:

def LinearRegressionModel(X_train, X_test, y_train, y_test):
  from sklearn.linear_model import LinearRegression
  from sklearn.metrics import mean_squared_error as MSE
  from sklearn.metrics import r2_score
  #Linear Regression
  model = LinearRegression() #Available Hyperparameter: Weights for particular input variable
  model.fit(X_train, y_train)
  ypred_train = model.predict(X_train)
  ypred_test = model.predict(X_test)
  ypred = concatenate((ypred_train, ypred_test), axis=0)
  y = concatenate((y_train,y_test), axis=0)
  train_accuracy = r2_score(y_train, ypred_train)
  test_accuracy = r2_score(y_test,ypred_test)
  RMSE_train = MSE(y_train,ypred_train)**0.5
  RMSE_test = MSE(y_test,ypred_test)**0.5
  overall_accuracy= r2_score(y, ypred)
  RMSE_overall = MSE(y,ypred)**0.5
  perf = pd.DataFrame(columns = ['Training R2', 'Testing R2','Overall R2', 'Training RMSE', 'Testing RMSE',  'Overall RMSE'],
        index = ['Linear Regression'])
  outputm = pd.DataFrame(columns = ['Linear Regression'])
  perf.loc['Linear Regression'] = [ train_accuracy, test_accuracy,overall_accuracy, RMSE_train,RMSE_test,RMSE_overall ]
  outputm['Linear Regression'] = ypred
  # plt.scatter(y,ypred)
  # print(perf)
  # print(outputm)
  return perf,outputm

In [ ]:
def SupportVectorMachineModel(X_train, X_test, y_train, y_test):
  from numpy.core.multiarray import concatenate
  #SVFLinear
  from sklearn.svm import SVR
  from sklearn.metrics import mean_squared_error as MSE
  from sklearn.metrics import r2_score
  from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

 #SVF Radial Basis Function
  # defining parameter range
  param_grid = {'C': [1, 10, 100],  #C is the l2 regularization paramter
              'gamma': [1],
              'epsilon':[0.003],
              'kernel': ['rbf']}

  model = GridSearchCV(SVR(), param_grid, refit = True, verbose = 1)
  model.fit(X_train, y_train)
  ypred_train = model.predict(X_train)
  ypred_test = model.predict(X_test)
  ypred = concatenate((ypred_train, ypred_test), axis=0)
  y = concatenate((y_train,y_test), axis=0)
  train_accuracy = r2_score(y_train, ypred_train)
  test_accuracy = r2_score(y_test,ypred_test)
  RMSE_train = MSE(y_train,ypred_train)**0.5
  RMSE_test = MSE(y_test,ypred_test)**0.5
  overall_accuracy= r2_score(y, ypred)
  RMSE_overall = MSE(y,ypred)**0.5
  perf = pd.DataFrame(columns = ['Training R2', 'Testing R2','Overall R2', 'Training RMSE', 'Testing RMSE',  'Overall RMSE'],
        index = ['SVM'])
  outputm = pd.DataFrame(columns = ['SVM'])
  perf.loc['SVM'] = [ train_accuracy, test_accuracy,overall_accuracy, RMSE_train,RMSE_test,RMSE_overall ]
  outputm['SVM'] = ypred
  #plt.scatter(y,ypred)
  #print(perf)
  #print(outputm)
  best_param = model.best_params_
  return perf,outputm, best_param

In [ ]:
def RandomForestModel(X_train, X_test, y_train, y_test):
  #RF
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.metrics import mean_squared_error as MSE
  from sklearn.metrics import r2_score
  from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
  # Instantiate model with 1000 decision trees
  model = GridSearchCV(RandomForestRegressor(random_state=seed),
                           {
                              'n_estimators':np.arange(90,100,5),
                              'max_features':np.arange(0.2,0.5,0.05),

                            },cv=5, scoring="r2",verbose=1,n_jobs=-1)
  model.fit(X_train, y_train)
  ypred_train = model.predict(X_train)
  ypred_test = model.predict(X_test)
  ypred = concatenate((ypred_train, ypred_test), axis=0)
  y = concatenate((y_train,y_test), axis=0)
  train_accuracy = r2_score(y_train, ypred_train)
  test_accuracy = r2_score(y_test,ypred_test)
  RMSE_train = MSE(y_train,ypred_train)**0.5
  RMSE_test = MSE(y_test,ypred_test)**0.5
  overall_accuracy= r2_score(y, ypred)
  RMSE_overall = MSE(y,ypred)**0.5
  perf = pd.DataFrame(columns = ['Training R2', 'Testing R2','Overall R2', 'Training RMSE', 'Testing RMSE',  'Overall RMSE'],
        index = ['RF'])
  outputm = pd.DataFrame(columns = ['RF'])
  perf.loc['RF'] = [ train_accuracy, test_accuracy,overall_accuracy, RMSE_train,RMSE_test,RMSE_overall ]
  outputm['RF'] = ypred
  # plt.scatter(y,ypred)
  # print(perf)
  # print(outputm)
  best_param = model.best_params_
  return perf,outputm, best_param


In [ ]:
def XGBoostModel(X_train, X_test, y_train, y_test):
  import xgboost as xgb
  from sklearn.metrics import mean_squared_error as MSE
  from sklearn.metrics import r2_score
  from sklearn.model_selection import GridSearchCV


  dtrain_reg = xgb.DMatrix(X_train, y_train)
  dtest_reg = xgb.DMatrix(X_test, y_test)

  # Define hyperparameters
  params = {"objective": "reg:squarederror"}

  n = 1000
  model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
  )
  ypred_train = model.predict(dtrain_reg)
  ypred_test = model.predict(dtest_reg)
  ypred = concatenate((ypred_train, ypred_test), axis=0)
  y = concatenate((y_train,y_test), axis=0)
  train_accuracy = r2_score(y_train, ypred_train)
  test_accuracy = r2_score(y_test,ypred_test)
  RMSE_train = MSE(y_train,ypred_train)**0.5
  RMSE_test = MSE(y_test,ypred_test)**0.5
  overall_accuracy= r2_score(y, ypred)
  RMSE_overall = MSE(y,ypred)**0.5
  perf = pd.DataFrame(columns = ['Training R2', 'Testing R2','Overall R2', 'Training RMSE', 'Testing RMSE',  'Overall RMSE'],
        index = ['XGB'])
  outputm = pd.DataFrame(columns = ['XGB'])
  perf.loc['XGB'] = [ train_accuracy, test_accuracy,overall_accuracy, RMSE_train,RMSE_test,RMSE_overall ]
  outputm['XGB'] = ypred
  # plt.scatter(y,ypred)
  # print(perf)
  # print(outputm)
  # print(params)
  return perf,outputm, params

In [ ]:
#Ozone
perflrO3,outputmlrO3 = LinearRegressionModel(X_train_1, X_test_1, O3_train, O3_test)
perfsvmO3,outputmsvmO3, psvmO3 = SupportVectorMachineModel(X_train_scaled, X_test_scaled, O3_train, O3_test)
perfrfO3,outputmrfO3, prfO3 = RandomForestModel(X_train_scaled, X_test_scaled, O3_train, O3_test)
perfxgbO3,outputmxgbO3, pxgbO3 = XGBoostModel(X_train_1, X_test_1, O3_train, O3_test)



#PM2.5
perflrPM25,outputmlrPM25 = LinearRegressionModel(X_train_1, X_test_1, PM25_train, PM25_test)
perfsvmPM25,outputmsvmPM25, psvmPM25 = SupportVectorMachineModel(X_train_scaled, X_test_scaled, PM25_train, PM25_test)
perfrfPM25,outputmrfPM25, prfPM25 = RandomForestModel(X_train_scaled, X_test_scaled,PM25_train, PM25_test)
perfxgbPM25,outputmxgbPM25, pxgbPM25 = XGBoostModel(X_train_1, X_test_1, PM25_train, PM25_test)



#PM10

perflrPM10,outputmlrPM10 = LinearRegressionModel(X_train_1, X_test_1, PM10_train, PM10_test)
perfsvmPM10,outputmsvmPM10, psvmPM10 = SupportVectorMachineModel(X_train_scaled, X_test_scaled, PM10_train, PM10_test)
perfrfPM10,outputmrfPM10, prfPM10 = RandomForestModel(X_train_scaled, X_test_scaled, PM10_train, PM10_test)
perfxgbPM10,outputmxgbPM10, pxgbPM10 = XGBoostModel(X_train_1, X_test_1, PM10_train, PM10_test)






Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
print(psvmO3)
print(prfO3)
print(pxgbO3)
print(psvmPM25)
print(prfPM25)
print(pxgbPM25)
print(psvmPM10)
print(prfPM10)
print(pxgbPM10)

{'C': 1, 'epsilon': 0.003, 'gamma': 1, 'kernel': 'rbf'}
{'max_features': 0.25, 'n_estimators': 95}
{'objective': 'reg:squarederror'}
{'C': 10, 'epsilon': 0.003, 'gamma': 1, 'kernel': 'rbf'}
{'max_features': 0.39999999999999997, 'n_estimators': 95}
{'objective': 'reg:squarederror'}
{'C': 100, 'epsilon': 0.003, 'gamma': 1, 'kernel': 'rbf'}
{'max_features': 0.25, 'n_estimators': 95}
{'objective': 'reg:squarederror'}


In [ ]:
perfallO3 = pd.concat([perflrO3, perfsvmO3, perfrfO3, perfxgbO3], axis =0)
outputallO3 = pd.concat([TargetO3, outputmlrO3, outputmsvmO3, outputmrfO3, outputmxgbO3], axis=1)
perfallPM25 = pd.concat([perflrPM25, perfsvmPM25, perfrfPM25, perfxgbPM25], axis =0)
outputallPM25 = pd.concat([TargetPM25, outputmlrPM25, outputmsvmPM25, outputmrfPM25, outputmxgbPM25], axis=1)
perfallPM10 = pd.concat([perflrPM10, perfsvmPM10, perfrfPM10, perfxgbPM10], axis =0)
outputallPM10 = pd.concat([TargetPM10, outputmlrPM10, outputmsvmPM10, outputmrfPM10, outputmxgbPM10], axis=1)
with pd.ExcelWriter('output.xlsx') as excel_writer:
  perfallO3.to_excel(excel_writer, sheet_name='PerformanceO3', index=True)
  outputallO3.to_excel(excel_writer, sheet_name='PredictedO3', index=False)
  perfallPM25.to_excel(excel_writer, sheet_name='PerformancePM25', index=True)
  outputallPM25.to_excel(excel_writer, sheet_name='PredictedPM25', index=False)
  perfallPM10.to_excel(excel_writer, sheet_name='PerformancePM10', index=True)
  outputallPM10.to_excel(excel_writer, sheet_name='PredictedPM10', index=False)


In [ ]:
#Model based on Log of Datasets
X_train_log = X_train_1.copy()
X_test_log = X_test_1.copy()
X_train_log['7-day Ozone Concentration'] = np.log2(X_train_1['7-day Ozone Concentration'])
X_train_log['7-day PM2.5 Concentration'] = np.log2(X_train_1['7-day PM2.5 Concentration'])
X_train_log['7-day PM10 Concentration'] = np.log2(X_train_1['7-day PM10 Concentration'])
X_test_log['7-day Ozone Concentration'] = np.log2(X_test_1['7-day Ozone Concentration'])
X_test_log['7-day PM2.5 Concentration'] = np.log2(X_test_1['7-day PM2.5 Concentration'])
X_test_log['7-day PM10 Concentration'] = np.log2(X_test_1['7-day PM10 Concentration'])
O3_train_log = np.log2(O3_train)
O3_test_log = np.log2(O3_test)
PM25_train_log = np.log2(PM25_train)
PM25_test_log = np.log2(PM25_test)
PM10_train_log = np.log2(PM10_train)
PM10_test_log = np.log2(PM10_test)
TargetO3_log = pd.DataFrame(columns = ['Date', 'Actual'])
TargetO3_log['Date'] = concatenate((TrainDate,TestDate), axis=0)
TargetO3_log['Actual'] = concatenate((O3_train_log,O3_test_log), axis=0)
TargetPM25_log = pd.DataFrame(columns = ['Date', 'Actual'])
TargetPM25_log['Date'] = concatenate((TrainDate,TestDate), axis=0)
TargetPM25_log['Actual'] = concatenate((PM25_train_log,PM25_test_log), axis=0)
TargetPM10_log = pd.DataFrame(columns = ['Date', 'Actual'])
TargetPM10_log['Date'] = concatenate((TrainDate,TestDate), axis=0)
TargetPM10_log['Actual'] = concatenate((PM10_train_log,PM10_test_log), axis=0)


In [ ]:
X_train_log

,7-day Ozone Concentration,7-day PM2.5 Concentration,7-day PM10 Concentration,Daily Average Dry Bulb Temperature,Daily Average Relative Humidity,Daily Average Wind Speed,Daily Peak Wind Direction,Daily Precipitation
1071,-5.351537,3.675281,4.520198,43,90,15.9,210,0.01
84,-4.609261,2.723271,4.701760,58,63,11.2,190,0.36
2399,-4.907951,3.100466,4.782483,66,86,11.5,70,0.00
776,-4.680717,2.618837,4.209188,25,30,8.5,30,0.00
841,-4.152989,3.114091,4.703080,66,54,9.0,320,0.02
...,...,...,...,...,...,...,...,...
1638,-4.358721,3.268110,4.727272,77,53,5.4,80,0.00
1095,-5.142582,2.795881,4.406478,32,54,13.2,230,0.03
1130,-4.597438,2.483834,4.028695,32,55,5.6,100,0.00
1294,-4.600685,3.858254,4.741724,72,78,10.5,10,0.13


In [ ]:
#Importing packages

from sklearn.preprocessing import StandardScaler

#Scaling
X_train_scaled_log = StandardScaler().fit_transform(X_train_log) #x_norm = (x-xmin)/(xmax-xmin) #x_norm= (x-xmean)/ x_std
X_test_scaled_log = StandardScaler().fit_transform(X_test_log)

In [ ]:
#Ozone
perflrO3_log,outputmlrO3_log = LinearRegressionModel(X_train_log, X_test_log, O3_train_log, O3_test_log)
perfsvmO3_log,outputmsvmO3_log, psvmO3_log = SupportVectorMachineModel(X_train_scaled_log, X_test_scaled_log, O3_train_log, O3_test_log)
perfrfO3_log,outputmrfO3_log, prfO3_log = RandomForestModel(X_train_scaled_log, X_test_scaled_log, O3_train_log, O3_test_log)
perfxgbO3_log,outputmxgbO3_log, pxgbO3_log = XGBoostModel(X_train_log, X_test_log, O3_train_log, O3_test_log)



#PM2.5
perflrPM25_log,outputmlrPM25_log = LinearRegressionModel(X_train_log, X_test_log, PM25_train_log, PM25_test_log)
perfsvmPM25_log,outputmsvmPM25_log, psvmPM25_log = SupportVectorMachineModel(X_train_scaled_log, X_test_scaled_log, PM25_train_log, PM25_test_log)
perfrfPM25_log,outputmrfPM25_log, prfPM25_log = RandomForestModel(X_train_scaled_log, X_test_scaled_log,PM25_train_log, PM25_test_log)
perfxgbPM25_log,outputmxgbPM25_log, pxgbPM25_log = XGBoostModel(X_train_log, X_test_log, PM25_train_log, PM25_test_log)



#PM10

perflrPM10_log,outputmlrPM10_log = LinearRegressionModel(X_train_log, X_test_log, PM10_train_log, PM10_test_log)
perfsvmPM10_log,outputmsvmPM10_log, psvmPM10_log = SupportVectorMachineModel(X_train_scaled_log, X_test_scaled_log, PM10_train_log, PM10_test_log)
perfrfPM10_log,outputmrfPM10_log, prfPM10_log = RandomForestModel(X_train_scaled_log, X_test_scaled_log, PM10_train_log, PM10_test_log)
perfxgbPM10_log,outputmxgbPM10_log, pxgbPM10_log = XGBoostModel(X_train_log, X_test_log, PM10_train_log, PM10_test_log)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
perfallO3_log = pd.concat([perflrO3_log, perfsvmO3_log, perfrfO3_log, perfxgbO3_log], axis =0)
outputallO3_log = pd.concat([TargetO3_log, outputmlrO3_log, outputmsvmO3_log, outputmrfO3_log, outputmxgbO3_log], axis=1)
perfallPM25_log = pd.concat([perflrPM25_log, perfsvmPM25_log, perfrfPM25_log, perfxgbPM25_log], axis =0)
outputallPM25_log = pd.concat([TargetPM25_log,outputmlrPM25_log, outputmsvmPM25_log, outputmrfPM25_log, outputmxgbPM25_log], axis=1)
perfallPM10_log = pd.concat([perflrPM10_log, perfsvmPM10_log, perfrfPM10_log, perfxgbPM10_log], axis =0)
outputallPM10_log = pd.concat([TargetPM10_log,outputmlrPM10_log, outputmsvmPM10_log, outputmrfPM10_log, outputmxgbPM10_log], axis=1)
with pd.ExcelWriter('output_log.xlsx') as excel_writer:
  perfallO3_log.to_excel(excel_writer, sheet_name='PerformanceO3_log', index=True)
  outputallO3_log.to_excel(excel_writer, sheet_name='PredictedO3_log', index=False)
  perfallPM25_log.to_excel(excel_writer, sheet_name='PerformancePM25_log', index=True)
  outputallPM25_log.to_excel(excel_writer, sheet_name='PredictedPM25_log', index=False)
  perfallPM10_log.to_excel(excel_writer, sheet_name='PerformancePM10_log', index=True)
  outputallPM10_log.to_excel(excel_writer, sheet_name='PredictedPM10_log', index=False)

In [ ]:
#Error Matrix = rmse, plot- residuals,
#Optimization plot- loss curve (Draft Report) # Try to find out from the worksheet 8.2
#Scatter plot with R2 values (Actual vs Predicted)
#Time series for predicted and actual- Draft Report
#

In [ ]:
#Copy the HW to the group scratch space
#Read the excel for Two layer Neural Network

#Deliverable:
#LR
#Model structure (Barua 2021)
#Explain the model parameters
#Plots
#Explain and discuss the results
#...
#Model Comparison
#Which model is best?
#Time series plot